In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.preprocessing import RobustScaler
from umap import UMAP
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters
from sklearn.metrics import silhouette_score
import glob
from itertools import groupby
from operator import itemgetter
from scipy.stats import mannwhitneyu
from figurefirst import mpl_functions

In [7]:
def get_index(data,th):
    idx = []
    for i in range(len(data)):
        if (data[i]>=th):
            idx.append(i)
    
    index = []
    for k, g in groupby(enumerate(idx),lambda ix : ix[0] - ix[1]):
        index.append((list((map(itemgetter(1), g)))))
    return index

def scale_data(data):
    min_val =  np.min(data)
    max_val = np.max(data)
    return (((data - min_val) / (max_val - min_val)) * 10)

def load_and_combine(file_pattern):
    """Load and combine trajectory files with proper ID handling."""
    files = sorted(glob.glob(file_pattern))
    dfs = []
    for i, f in enumerate(files):
        df = pd.read_hdf(f)
        df = df.iloc[:-1]  # Drop last row
        df['trajectory_id'] = f'traj_{i}'
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

def load_and_combine_cfd(file_pattern):
    """Load and combine trajectory files with proper ID handling."""
    files = sorted(glob.glob(file_pattern))
    dfs = []
    for i, f in enumerate(files):
        df = pd.read_hdf(f)
        df = df.iloc[:-1]  # Drop last row
        df['trajectory_id'] = f'traj_{i}'
        dfs.append(df)
    combined_df=pd.concat(dfs, ignore_index=True)
    combined_df['odor']=scale_data(combined_df.odor)
    return combined_df

def calculate_metrics(X_umap, labels):
    """Calculate silhouette score and normalized centroid distance."""
    # Convert string labels to numeric for silhouette score
    unique_labels = np.unique(labels)
    label_map = {label: i for i, label in enumerate(unique_labels)}
    numeric_labels = np.array([label_map[label] for label in labels])
    
    # Calculate silhouette score
    silhouette = silhouette_score(X_umap, numeric_labels)
    
    # Calculate centroids
    centroids = []
    for label in range(len(unique_labels)):
        mask = numeric_labels == label
        centroid = np.mean(X_umap[mask], axis=0)
        centroids.append(centroid)
    centroids = np.array(centroids)
    
    # Calculate distance between centroids
    centroid_dist = cdist(centroids, centroids)[0, 1]
    
    # Calculate average spread within clusters
    spreads = []
    for label, centroid in enumerate(centroids):
        mask = numeric_labels == label
        points = X_umap[mask]
        spread = np.mean(cdist(points, centroid.reshape(1, -1)))
        spreads.append(spread)
    
    avg_spread = np.mean(spreads)
    normalized_dist = centroid_dist / (avg_spread + 1e-10)
    
    return silhouette, normalized_dist

def process_odor_signal(df, threshold=6.5):
    """Process odor signal to keep only whiff values."""
    # Create a copy of the odor column
    df = df.copy()    
    # Process odor by trajectory
    for traj_id, traj_df in df.groupby('trajectory_id'):
        odor_series = traj_df['odor'].values
        
        # Get whiff indices
        whiff_idx = get_index(odor_series, threshold)
        
        # Create mask of zeros
        processed_odor = np.zeros_like(odor_series)
        
        # Set whiff values
        for idx_group in whiff_idx:
            processed_odor[idx_group] = odor_series[idx_group]
        
        # Update the dataframe
        df.loc[traj_df.index, 'odor'] = processed_odor
    return df



In [ ]:
class TrajectoryComparison:
    def __init__(self, cosmos_df, cfd_df):
        self.cosmos_df = cosmos_df
        self.cfd_df = cfd_df
        self.feature_scaler = None
        self.umap_reducer = None
        
    def normalize_units(self):
        """Ensure consistent units between datasets"""
        # Scale spatial coordinates to same range
        for df in [self.cosmos_df, self.cfd_df]:
            # Normalize spatial components
            spatial_cols = ['x', 'y', 'vx', 'vy', 'crosswind_dist', 'upwind_dist']
            for col in spatial_cols:
                if col in df.columns:
                    df[col] = df[col] / df[col].abs().max()
            
            # Normalize time steps
            df['time'] = (df['time'] - df['time'].min()) / (df['time'].max() - df['time'].min())
            
            # Normalize angles to [-1, 1]
            angle_cols = ['heading_angle']
            for col in angle_cols:
                if col in df.columns:
                    df[col] = np.unwrap(df[col]) % (2 * np.pi)
                    df[col] = df[col] / np.pi - 1
            
            # Normalize other quantities
            if 'speed' in df.columns:
                df['speed'] = df['speed'] / df['speed'].max()
            if 'acceleration' in df.columns:
                df['acceleration'] = df['acceleration'] / df['acceleration'].abs().max()
            if 'angular_velocity' in df.columns:
                df['angular_velocity'] = df['angular_velocity'] / df['angular_velocity'].abs().max()
            if 'path_curvature' in df.columns:
                df['path_curvature'] = df['path_curvature'] / df['path_curvature'].abs().max()
    
    def add_derived_features(self, df):
        """Add carefully normalized derived features"""
        # Simple path efficiency (ratio of displacement to path length)
        df['cumulative_dx'] = df.groupby('trajectory_id')['vx'].cumsum()
        df['cumulative_dy'] = df.groupby('trajectory_id')['vy'].cumsum()
        df['path_length'] = np.sqrt(df['cumulative_dx']**2 + df['cumulative_dy']**2)
        df['path_length'] = df['path_length'] / df['path_length'].max()
        
        # Turn rate (normalized)
        df['turn_rate'] = np.gradient(df['heading_angle'], df['time'])
        df['turn_rate'] = df['turn_rate'] / df['turn_rate'].abs().max()
        
        return df
    
    def extract_features(self):
        """Extract features using minimal tsfresh parameters"""
        # Define basic columns to extract features from
        base_cols = [
            "vx", "vy", "speed", "acceleration",
            "heading_angle", "angular_velocity",
            "crosswind_dist", "upwind_dist",
            "path_curvature", "turn_rate",
            "path_length"
        ]
        
        # Use only columns that exist in both datasets
        available_cols = []
        for col in base_cols:
            if col in self.cosmos_df.columns and col in self.cfd_df.columns:
                available_cols.append(col)
        
        # Use minimal feature set from tsfresh
        settings = MinimalFCParameters()
        # settings = EfficientFCParameters()
        
        # Extract features for both datasets
        cosmos_features = extract_features(
            self.cosmos_df[['trajectory_id', 'time'] + available_cols],
            column_id='trajectory_id',
            column_sort='time',
            default_fc_parameters=settings
        )
        
        cfd_features = extract_features(
            self.cfd_df[['trajectory_id', 'time'] + available_cols],
            column_id='trajectory_id',
            column_sort='time',
            default_fc_parameters=settings
        )
        
        # Ensure both feature sets have the same columns
        common_cols = cosmos_features.columns.intersection(cfd_features.columns)
        cosmos_features = cosmos_features[common_cols]
        cfd_features = cfd_features[common_cols]
        
        # Balance datasets if needed
        min_size = min(len(cosmos_features), len(cfd_features))
        if len(cosmos_features) > min_size:
            cosmos_features = cosmos_features.sample(n=min_size, random_state=42)
        if len(cfd_features) > min_size:
            cfd_features = cfd_features.sample(n=min_size, random_state=42)
        
        return cosmos_features, cfd_features
    
    def fit_transform(self, n_neighbors=15, min_dist=0.1, metric='euclidean'):
        """Fit and transform data with careful preprocessing"""
        # Normalize units
        self.normalize_units()
        
        # Add derived features
        self.cosmos_df = self.add_derived_features(self.cosmos_df)
        self.cfd_df = self.add_derived_features(self.cfd_df)
        
        # Extract features
        cosmos_features, cfd_features = self.extract_features()
        
        # print("Number of features:", len(cosmos_features.columns))
        # print("\nFeature names:")
        # for col in cosmos_features.columns:
        #     print(col)


        
        # Combine features
        X = pd.concat([cosmos_features, cfd_features])
        y = np.concatenate([
            np.zeros(len(cosmos_features)),
            np.ones(len(cfd_features))
        ])
        
        # Scale features
        self.feature_scaler = RobustScaler()
        X_scaled = self.feature_scaler.fit_transform(X)
        
        # Fit UMAP
        self.umap_reducer = UMAP(
            n_neighbors=n_neighbors,
            min_dist=min_dist,
            metric=metric,
            random_state=42
        )
        X_umap = self.umap_reducer.fit_transform(X_scaled)
        
        return X_umap, y
        
    def analyze_temporal_correlation(self):
        """Analyze temporal correlation between odor encounters and movement patterns."""
        def compute_windowed_correlation(odor, velocity, window_size=100):
            T = len(odor)
            correlation = np.zeros(T)
            
            # Standardize signals
            odor_std = (odor - np.mean(odor)) / np.std(odor)
            vel_std = (velocity - np.mean(velocity)) / np.std(velocity)
            
            # Compute windowed correlation
            for t in range(T - window_size):
                odor_window = odor_std[t:t+window_size]
                vel_window = vel_std[t:t+window_size]
                correlation[t] = np.sum(odor_window * vel_window) / window_size
                
            return correlation

        # Compute correlations for both simulators
        cosmos_correlations = []
        cfd_correlations = []
        
        # Process COSMOS trajectories
        for traj_id in self.cosmos_df['trajectory_id'].unique():
            traj = self.cosmos_df[self.cosmos_df['trajectory_id'] == traj_id]
            odor = traj['odor'].values
            velocity = np.sqrt(traj['vx']**2 + traj['vy']**2).values
            corr = compute_windowed_correlation(odor, velocity)
            cosmos_correlations.append(np.mean(corr))
        
        # Process CFD trajectories
        for traj_id in self.cfd_df['trajectory_id'].unique():
            traj = self.cfd_df[self.cfd_df['trajectory_id'] == traj_id]
            odor = traj['odor'].values
            velocity = np.sqrt(traj['vx']**2 + traj['vy']**2).values
            corr = compute_windowed_correlation(odor, velocity)
            cfd_correlations.append(np.mean(corr))
        
        # Statistical test
        statistic, pvalue = mannwhitneyu(cosmos_correlations, cfd_correlations)
        
        return {
            'cosmos_correlations': np.array(cosmos_correlations),
            'cfd_correlations': np.array(cfd_correlations),
            'p_value': pvalue
        }
    
    def plot_comparison(self, X_umap, labels, silhouette, norm_dist):
        """Plot comparison with enhanced visualization and axis labels."""
        f, ax = plt.subplots(figsize=(5, 5))
        
        # Plot points
        for i, label in enumerate(['COSMOS', 'CFD']):
            mask = labels == i
            ax.scatter(
                X_umap[mask, 0],
                X_umap[mask, 1],
                label=label,
                alpha=0.6,
                s=50,
                c=['#bc141a', '#1764ab'][i]
            )
        
        # Calculate variance ratios directly from UMAP coordinates
        # var_ratio = np.var(X_umap, axis=0)
        # var_ratio = (var_ratio / np.sum(var_ratio)) * 100
        
        # # Add labels with variance percentages
        # ax.set_xlabel(f'UMAP 1 ({var_ratio[0]:.1f}% var)')
        # ax.set_ylabel(f'UMAP 2 ({var_ratio[1]:.1f}% var)')
        
        ax.set_xlabel('UMAP 1')
        ax.set_ylabel('UMAP 2')
        mpl_functions.adjust_spines(ax,['left', 'bottom'],
                           spine_locations={'left': 5, 'bottom': 5}, 
                           smart_bounds=True,
                           yticks=[-4,8],
                           xticks=[-5,15],
                           linewidth=0.5)

        ax.set_title(f'$a_s$={silhouette:.2f} $d_{{norm}}$={norm_dist:.2f}')
        ax.legend()
        mpl_functions.set_fontsize(ax, 14)
        f.tight_layout()
        
        return f

if __name__ == "__main__":
    path = "data/tracking/trajectories/"
    osdf_combined = load_and_combine(str(path)+"cosmos*.h5")
    cfd_df_combined = load_and_combine_cfd(str(path)+"cfd*.h5")
    
    # Create comparison object
    comparator = TrajectoryComparison(osdf_combined, cfd_df_combined)
    X_umap, labels = comparator.fit_transform(
        n_neighbors=50,  # 
        min_dist=0.1,    
        metric='euclidean'  
    )
    
    silhouette, norm_dist = calculate_metrics(X_umap, labels)
    comparator.plot_comparison(X_umap, labels, silhouette, norm_dist)

    print("\nCluster Quality Metrics:")
    print(f"Silhouette Score: {silhouette:.3f}")
    print(f"Normalized Centroid Distance: {norm_dist:.3f}")
    
# Analyze temporal correlations
    # correlation_results = comparator.analyze_temporal_correlation()

    # print(f"Correlation Analysis p-value: {correlation_results['p_value']:.3f}")

    # # Optional: Plot correlation distributions
    # plt.figure(figsize=(6, 4))
    # plt.boxplot([correlation_results['cosmos_correlations'], 
    #             correlation_results['cfd_correlations']],
    #             labels=['COSMOS', 'CFD'])
    # plt.ylabel('Odor-Motion Correlation')
    # plt.title('Temporal Correlation Comparison')
    # plt.show()